In [7]:
!pip install scikit-learn
!pip install scipy

     |████████████████████████████████| 10.9 MB 4.5 MB/s eta 0:00:01    |████████████▍                   | 4.2 MB 4.5 MB/s eta 0:00:02
     |████████████████████████████████| 36.5 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 301 kB 18.4 MB/s eta 0:00:01


In [8]:
from sklearn.datasets import fetch_kddcup99

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, array, min as smin
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Creating a Spark session

In [2]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("k-meaner") \
    .config("spark.executor.memory", "1024m") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/06 07:17:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Loading the `kddcup99` dataset

In [ ]:
kdd = spark.createDataFrame(fetch_kddcup99(as_frame=True)["frame"])

/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


# Pre-processing
We remove the first three “categorical” columns, and keep only the numerical ones. Then we rescale those to a [0, 1] range. Also, we remove two columns that happen to be all zeros. After all, this shouldn’t affect the clustering afterwards.

In [ ]:
kdd.printSchema()

In [ ]:
kdd.createOrReplaceTempView("kdd_table")

In [ ]:
for c in kdd.columns:
    try:
        limits = spark.sql("SELECT min(" + c + "), max(" + c + ") FROM kdd_table").collect()
        #mx = kdd.groupby().max(c).first().asDict()['max('+c+')']
        #mn = kdd.groupby().min(c).first().asDict()['min('+c+')']
        mn, mx = limits[0][0], limits[0][1]
        kdd = kdd.withColumn(c, (col(c)-mn)/(mx-mn) )
        #print(limits[0][0], limits[0][1])
    except Exception as e:
        print(e)

#kdd.take(1)
#help(kdd)

assembler = VectorAssembler(inputCols=[c for c in kdd.columns if c not in ["protocol_type", "service", "flag", "labels"]], outputCol="features")
assembled_df = assembler.transform(kdd)

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(assembled_df)
normalized_data = scaler_model.transform(assembled_df)

### Plotting the distribution of attack types…

In [ ]:
"""targets = kdd[1]

attacks, counts = np.unique(targets, return_counts=True)
ypos = np.arange(len(attacks))

attack_sort = np.argsort(counts)
attacks = attacks[attack_sort]
counts = counts[attack_sort]
"""
res = spark.sql("SELECT labels, count(1) FROM kdd_table GROUP BY labels").collect()

In [ ]:
attacks = np.array([r[0] for r in res])
counts = np.array([r[1] for r in res])
attack_sort = np.argsort(counts)
attacks = attacks[attack_sort]
counts = counts[attack_sort]

ypos = np.arange(len(attacks))

In [ ]:
plt.figure(figsize=(12, 6))
plt.barh(ypos, np.log10(counts), facecolor="gold")
plt.yticks(ypos, labels=[a.decode("utf-8")[0:-1] for a in attacks])
plt.xlabel(r"$\log_{10}(\mathrm{count})$")
plt.ylabel("Attack type")
plt.show()

## Start on k-means basic algo

In [ ]:
ignored_cols = ["protocol_type", "service", "flag", "labels", "centr"]
ncols = len(kdd.columns) - len(ignored_cols)

def dist(x,y):
    try:
        return (((np.array(x)-y)**2).sum())**0.5
    except:
        return ncols

def argcomp(comp,func, arr, *params):
    res = [func(arr[i], *params) for i in range(len(arr))]
    #print(res)
    return res.index(comp(res))

#udist=udf(dist, FloatType())

#distance_udf = udf(lambda x,y:  np.linalg.norm(x-y), FloatType())

###################################### actual k-means 
def kmeans(data, centers):
    count = 0
    while True:
        #data = data.map(lambda x: [argcomp(min,dist,centers,x), x[1]])
        #newcenters = data.reduceByKey(lambda x,y: np.mean([x,y], axis=1))
        cc = centers.collect()
        #print(cc)
        argmindist_udf = udf(lambda row: argcomp(min,dist,cc,row), IntegerType())
        
        #data = data.withColumn("centr", argcomp(min, dist, centers.collect(), [data[c] for c in data.columns if c not in ignored_cols]))
        data = data.withColumn("centr", argmindist_udf(array([c for c in data.columns if c not in ignored_cols])))
        
        centr = data.select("centr").where(data.centr!=0).collect()
        if len(centr) == 0:
            print("!!!! all zero !!!!")
        
        newcenters = data.groupBy("centr").mean().select(*[col("avg("+c+")") for c in data.columns if c not in ignored_cols])
        
        print(count)
        count+=1
        if count>1:#(newcenters-centers).mean() < 0.01:
            break
        else:
            centers = newcenters
    
    return data, centers

In [ ]:
centers = pd.DataFrame(np.random.random(size=(ncols,len(attacks))))
centers = spark.createDataFrame(centers)

In [ ]:
finaldata, newcenters = kmeans(kdd, centers)

In [ ]:
newcenters.collect()

In [ ]:
centr = finaldata.select("centr").collect()

for i in range(len(centr)):
    if(centr[i][0] != 0):
        print(centr[i][0])

In [ ]:
test_data = pd.DataFrame(np.random.random(size=(30,2))*2-1)
test_data["labels"]=[*([0]*30)]
test_data["protocol_type"]=[*(["http"]*30)]
test_data["protocol_type"][3]="smtp"

In [ ]:
test_data["labels"][(test_data[0]<0) & (test_data[1]>0)]=1
test_data["labels"][(test_data[0]<0) & (test_data[1]<0)]=2
test_data["labels"][(test_data[0]>0) & (test_data[1]<0)]=3

In [ ]:
test_data

In [ ]:
test_data = spark.createDataFrame(test_data)
#test_cent = spark.createDataFrame(pd.DataFrame(np.array([[1,1], [-1,1], [-1,-1], [1,-1]])))
test_cent = spark.createDataFrame(pd.DataFrame(np.random.random(size=(4,2))*2-1))

In [ ]:
res_tdata, res_tcent = kmeans(test_data, test_cent)

In [ ]:
res_tcent.collect()

In [ ]:
d = res_tdata.collect()

In [ ]:
#print(d, len(d))
mat = [[0 for i in range(4)] for j in range(4)]
for i in range(4):
    for j in range(4):
        for k in range(len(d)):
            mat[i][j] += ((d[k]["labels"] == i) and (d[k]["centr"] == j))
mat